In [1]:
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END, add_messages
from langgraph.types import Command
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolCallId
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv
from langsmith import traceable
import asyncio

load_dotenv()

memory = MemorySaver()


class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    result: int

In [2]:
@tool
async def operate(
    a: int,
    b: int,
    tool_call_id: Annotated[str, InjectedToolCallId],
) -> Command:
    """Perform operation on 2 numbers"""

    await asyncio.sleep(5)
    return Command(
        update={
            "messages": [
                ToolMessage(
                    f"Operation performed successfully", tool_call_id=tool_call_id
                )
            ],
            "result": a + b,
        }
    )

In [7]:
tools = [operate]

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools=tools)

tool_node = ToolNode(tools=tools)


@traceable
async def model_node(state: AgentState) -> AgentState:
    result = await model.ainvoke(state["messages"])
    return {**state, "messages": [result]}


@traceable
async def tools_router(state: AgentState):
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool_node"

    return END


graph_builder = StateGraph(AgentState)

graph_builder.add_node("model", model_node)
graph_builder.add_node("tool_node", tool_node)

graph_builder.set_entry_point("model")

graph_builder.add_conditional_edges("model", tools_router)
graph_builder.add_edge("tool_node", "model")

graph = graph_builder.compile(checkpointer=memory)


async for event in graph.astream_events(
    {
        "messages": [HumanMessage(content="Perform operation on 10 and 4")],
        "result": 0,
    },
    {"configurable": {"thread_id": 11}},
):
    print(event)

# await graph.ainvoke(
#     {
#         "messages": [HumanMessage(content="Perform operation on 2 and 4")],
#         "result": 0,
#     },
#     {"configurable": {"thread_id": 11}},
# )

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Perform operation on 10 and 4', additional_kwargs={}, response_metadata={})], 'result': 0}}, 'name': 'LangGraph', 'tags': [], 'run_id': '65cba7de-eebb-441a-b848-986235f65328', 'metadata': {'thread_id': 11}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Perform operation on 5 and 4', additional_kwargs={}, response_metadata={}, id='e03b0566-ebc2-4723-bf77-0a407235fd7b'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_bd3B2g6gA4nOFQjvamGMORpo', 'function': {'arguments': '{"a":5,"b":4}', 'name': 'operate'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--e2d38e95-b2ea-4284-bd26-0f9d882bd3b4', tool_calls=[{'name': 'operate', 'args': {'a': 5, 'b': 4}, 'id': 'call_bd3B2g6gA4nOFQjvamGMORp